In [3]:
import pandas as pd
import duckdb
from utils.quack import Quack
pd.set_option('display.max_columns', 45)

In [4]:
with open("./data/queries/player_feature_eng.sql", "r") as f:
    sql = f.read()
con = duckdb.connect("data/nfl.duckdb")
df = con.execute(sql).fetchdf()

In [5]:
df.loc[(df.season==2024) &
                  (df.week==4) &
                  (df.team=='MIA')]

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,...,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr,player_id,player_display_name,opponent,offense_snaps,offense_pct
72046,0,MIA,TE,00-0034798,2024_04_TEN_MIA,2024,4,2024,Durham,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0034798,Durham,TEN,18.0,0.30
72047,1,MIA,TE,00-0038720,2024_04_TEN_MIA,2024,4,2024,Julian,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0038720,Julian,TEN,30.0,0.50
72065,0,MIA,RB,00-0039040,2024_04_TEN_MIA,2024,4,2024,Devon,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,14.0,0.0,0.0,0.0,-6.0,20.0,0.0,-1.009796,0.0,-2.333333,0.142857,-0.034091,0.190422,0.0,2.90,5.90,00-0039040,De'Von Achane,TEN,44.0,0.73
72066,1,MIA,RB,00-0039874,2024_04_TEN_MIA,2024,4,2024,Jaylen,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,-0.430790,0.0,-0.000000,0.047619,-0.005682,0.067451,0.0,3.20,3.20,00-0039874,Jaylen Wright,TEN,28.0,0.47
72067,2,MIA,RB,00-0034115,2024_04_TEN_MIA,2024,4,2024,Jeffery,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00-0034115,Jeffery,TEN,2.0,0.03
72085,0,MIA,QB,00-0035993,2024_04_TEN_MIA,2024,4,2024,Tyler,1,14.0,22.0,96.0,0.0,0.0,2.0,-18.0,0.0,0.0,176.0,55.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.0,11.84,11.84,00-0035993,Tyler Huntley,TEN,60.0,1.00
72086,0,MIA,WR,00-0036613,2024_04_TEN_MIA,2024,4,2024,Jaylen,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,6.0,36.0,0.0,0.0,0.0,37.0,23.0,2.0,1.077499,0.0,0.972973,0.285714,0.210227,0.575731,0.0,3.60,7.60,00-0036613,Jaylen Waddle,TEN,43.0,0.72
72087,1,MIA,WR,00-0033040,2024_04_TEN_MIA,2024,4,2024,Tyreek,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7.0,23.0,0.0,0.0,0.0,129.0,4.0,1.0,-3.426462,0.0,0.178295,0.333333,0.732955,1.013068,0.0,4.20,8.20,00-0033040,Tyreek Hill,TEN,44.0,0.73
72088,2,MIA,WR,00-0034419,2024_04_TEN_MIA,2024,4,2024,Braxton,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.000000,0.000000,0.000000,0.0,0.00,0.00,00-0034419,Braxton Berrios,TEN,15.0,0.25
72089,3,MIA,WR,00-0039880,2024_04_TEN_MIA,2024,4,2024,Malik,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,8.0,0.0,0.0,0.0,5.0,3.0,0.0,0.056446,0.0,1.600000,0.047619,0.028409,0.091315,0.0,0.80,1.80,00-0039880,Malik Washington,TEN,14.0,0.23



depth = Quack.query("""select depth.*, snps.""")
off = depth.loc[depth.formation=='Offense'].copy()
#off.loc[off.offense_pct > 0]
depth

In [6]:
context_cols = ['player_id', 'player_display_name','position',
]
game_context = ['week','season_type', 'season', 'opponent_team', 'recent_team', 'game_id', 'game_type']
stat_cols = ['completions', 'attempts',
       'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost',
       'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards',
       'passing_yards_after_catch', 'passing_first_downs', 'passing_epa',
       'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards',
       'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa',
       'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share',
       'wopr', 'special_teams_tds', 'fantasy_points', 'fantasy_points_ppr',]
rush_stat_cols = [
     'carries', 'rushing_yards',
       'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost',
       'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions',
       'receptions', 'targets', 'receiving_yards', 'receiving_tds',
       'fantasy_points', 'fantasy_points_ppr',
       'offense_snaps','offense_pct'
]

depth_cols = ['season', 'last_name',
       'first_name', 'football_name', 'formation', 'gsis_id', 'jersey_number',
       'position', 'elias_id', 'depth_position', 'player_display_name']


In [7]:

qb_stats_eng = [
    "completions",
    "attempts",
    "passing_yards",
    "passing_tds",
    "passing_interceptions",
    "sacks_suffered",
    "sack_yards_lost",
    "passing_air_yards",
    "passing_yards_after_catch",
    "passing_first_downs",
    "passing_2pt_conversions",
    "carries",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
       #"completion_pct"
]

rush_stats_eng = [
    "carries",
    "rushing_yards",
    "rushing_tds",
    "rushing_first_downs",
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "fantasy_points",
    "fantasy_points_ppr",
       'offense_snaps','offense_pct',
      # 'yards_per_carry',
]
receiver_stats_eng = [
    "receptions",
    "targets",
    "receiving_yards",
    "receiving_tds",
    "receiving_air_yards",
    "receiving_yards_after_catch",
    "receiving_first_downs",
    "target_share",
    "air_yards_share",
    "fantasy_points_ppr",
      'offense_snaps','offense_pct'
]
stats_to_eng = list(set(qb_stats_eng + rush_stats_eng + receiver_stats_eng))

In [8]:
temp = df.copy().sort_values(by=['season','week']).reset_index(drop=True)

temp = off.merge(
    weekly,
    how="outer",
    left_on=["gsis_id", "team", "season", "week", "position", "game_id"],
    right_on=["player_id", "team", "season", "week", "position","game_id"],
).sort_values(by=['season','week']).reset_index(drop=True)
temp[context_cols] = temp.groupby(depth_cols)[context_cols].bfill()
temp[context_cols] = temp.groupby(depth_cols)[context_cols].ffill()


In [9]:
temp.loc[~temp.offense_snaps.isna(), stat_cols] = temp.loc[~temp.offense_snaps.isna(), stat_cols].fillna(0)
player_fantasy = temp.copy(deep=True).sort_values(by=['season','week']) 
roll_columns = [x + '_roll' for x in stats_to_eng]
player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[stats_to_eng].transform(lambda x: x.ewm(span=7, min_periods=0).mean().shift())
player_fantasy[roll_columns] = player_fantasy.groupby(context_cols)[roll_columns].shift(1)

In [10]:
player_fantasy['completion_pct_roll'] = player_fantasy['completions_roll'] / player_fantasy['attempts_roll']
player_fantasy['yards_per_carry_roll'] = (3 + player_fantasy['rushing_yards_roll']) / (1 + player_fantasy['carries_roll'])
player_fantasy['yards_per_catch_roll'] = (4 + player_fantasy['receiving_yards_roll']) / (1 + player_fantasy['receptions_roll'])
player_fantasy['yards_per_target_roll'] = (7 + player_fantasy['receiving_yards_roll']) / (1 + player_fantasy['targets_roll'])
receiver_stats_eng += ['yards_per_catch','yards_per_target']
rush_stats_eng += ['yards_per_carry']
qb_stats_eng += ['completion_pct']

In [11]:
player_fantasy.loc[player_fantasy.player_display_name =='Brock Bowers'].sort_values(by=['season','week'])#[['season','week','offense_snaps', 'receptions', 'fantasy_points_ppr', 'yards_per_catch_roll', 'receptions_roll']]

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,...,offense_pct_roll,passing_first_downs_roll,passing_tds_roll,rushing_first_downs_roll,passing_air_yards_roll,receiving_yards_after_catch_roll,receptions_roll,sack_yards_lost_roll,receiving_air_yards_roll,receiving_tds_roll,passing_yards_after_catch_roll,fantasy_points_roll,fantasy_points_ppr_roll,passing_yards_roll,offense_snaps_roll,rushing_yards_roll,carries_roll,rushing_tds_roll,completion_pct_roll,yards_per_carry_roll,yards_per_catch_roll,yards_per_target_roll
70944,1,LV,TE,00-0039338,2024_01_LV_LAC,2024,1,2024,Brock,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71476,1,LV,TE,00-0039338,2024_02_LV_BAL,2024,2,2024,Brock,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71678,1,LV,TE,00-0039338,2024_03_CAR_LV,2024,3,2024,Brock,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.680000,0.0,0.0,0.000000,0.0,16.000000,6.000000,0.0,39.000000,0.000000,0.0,5.800000,11.800000,0.0,41.000000,0.000000,0.000000,0.0,NaN,3.000000,8.857143,7.222222
71961,0,LV,TE,00-0039338,2024_04_CLE_LV,2024,4,2024,Brock,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.680000,0.0,0.0,0.000000,0.0,30.857143,7.714286,0.0,48.714286,0.000000,0.0,8.085714,15.800000,0.0,42.142857,0.000000,0.000000,0.0,NaN,3.000000,9.737705,9.179104
72577,0,LV,TE,00-0039338,2024_05_LV_DEN,2024,5,2024,Brock,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.671351,0.0,0.0,0.000000,0.0,27.459459,5.675676,0.0,37.162162,0.000000,0.0,6.362162,12.037838,0.0,42.081081,0.000000,0.000000,0.0,NaN,3.000000,10.129555,9.298932
72798,0,LV,TE,00-0039338,2024_06_PIT_LV,2024,6,2024,Brock,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.722057,0.0,0.0,0.365714,0.0,23.634286,4.331429,0.0,30.520000,0.000000,0.0,5.169143,9.500571,0.0,44.245714,4.388571,0.365714,0.0,NaN,5.410042,9.622722,8.646952
73246,0,LV,TE,00-0039338,2024_07_LV_LA,2024,7,2024,Brock,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.744328,0.0,0.0,0.245839,0.0,36.537772,5.533931,0.0,47.394366,0.327785,0.0,8.620999,14.154930,0.0,47.770807,2.950064,0.245839,0.0,NaN,4.775951,10.344895,8.321962
73399,0,LV,TE,00-0039338,2024_08_KC_LV,2024,8,2024,Brock,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.758218,0.0,0.0,0.171072,0.0,39.111375,6.588061,0.0,49.099198,0.228096,0.0,8.158420,14.746481,0.0,48.448767,2.052866,0.171072,0.0,NaN,4.314735,9.204666,7.876650
73773,0,LV,TE,00-0039338,2024_09_LV_CIN,2024,9,2024,Brock,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.781813,0.0,0.0,0.121716,0.0,40.810312,7.572445,0.0,53.686765,0.162288,0.0,8.487779,16.060224,0.0,53.512503,1.460590,0.121716,0.0,NaN,3.976578,9.061584,7.396585
74660,0,LV,TE,00-0039338,2024_11_LV_MIA,2024,11,2024,Brock,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.817425,0.0,0.0,0.087902,0.0,37.529258,6.857787,0.0,46.828470,0.117202,0.0,7.741080,14.598867,0.0,53.370123,1.054820,0.087902,0.0,NaN,3.727193,9.331361,7.996800


In [12]:
player_fantasy =player_fantasy.groupby(['team','season','week','game_id', 'position', 'dense_depth', 'gsis_id']).head(1)

In [13]:
con = duckdb.connect("data/nfl.duckdb")

player_weekly_agg_raw = player_fantasy[[x for x in player_fantasy.columns if x not in stat_cols]]
player_weekly_agg_raw.to_parquet("data/agg/player_weekly_agg.parquet")
con.execute("CREATE OR REPLACE TABLE player_weekly_agg AS SELECT * FROM player_weekly_agg_raw")


In [14]:
agg = player_fantasy.copy()
agg = agg.sort_values(by=['team','season','week','player_display_name'])
agg = agg.loc[~((agg.gsis_id == '00-0027874') & (agg.game_id == '2018_09_HOU_DEN'))]
agg

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,completions,attempts,passing_yards,passing_tds,passing_interceptions,sacks_suffered,sack_yards_lost,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,...,offense_pct_roll,passing_first_downs_roll,passing_tds_roll,rushing_first_downs_roll,passing_air_yards_roll,receiving_yards_after_catch_roll,receptions_roll,sack_yards_lost_roll,receiving_air_yards_roll,receiving_tds_roll,passing_yards_after_catch_roll,fantasy_points_roll,fantasy_points_ppr_roll,passing_yards_roll,offense_snaps_roll,rushing_yards_roll,carries_roll,rushing_tds_roll,completion_pct_roll,yards_per_carry_roll,yards_per_catch_roll,yards_per_target_roll
36,3,ARI,RB,00-0027346,2013_01_ARI_STL,2013,1,2013,Alfonso,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,2,ARI,RB,00-0030287,2013_01_ARI_STL,2013,1,2013,Andre,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
206,2,ARI,WR,00-0027691,2013_01_ARI_STL,2013,1,2013,Andre,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,0,ARI,QB,00-0021429,2013_01_ARI_STL,2013,1,2013,Carson,0,26.0,40.0,327.0,2.0,1.0,4.0,-23.0,2.0,1.0,385.0,61.0,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,2,ARI,TE,00-0030291,2013_01_ARI_STL,2013,1,2013,D.C.,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82450,1,WAS,QB,00-0032268,2025_14_WAS_MIN,2025,14,2025,Marcus Mariota,1,2.0,4.0,30.0,0.0,1.0,1.0,-8.0,0.0,0.0,54.0,1.0,2.0,...,0.725149,6.719952,0.981389,1.003957,173.137968,0.000005,4.810651e-07,-8.840937,0.000005,3.360107e-08,63.624254,11.829036,11.829036,149.488182,43.419456,27.768789,3.767390,0.069625,0.670475,6.454012,4.000008,7.000006
82455,4,WAS,WR,00-0033591,2025_14_WAS_MIN,2025,14,2025,Noah Brown,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.598717,0.000000,0.000000,0.000000,0.000000,8.007113,3.012785e+00,0.000000,54.402054,4.910869e-02,0.000000,3.904449,6.917234,0.000000,41.920504,-0.000594,0.000595,0.000000,NaN,2.997623,9.992797,7.157489
82451,0,WAS,WR,00-0035659,2025_14_WAS_MIN,2025,14,2025,Terry McLaurin,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.712953,0.000000,0.000000,0.000001,0.000000,16.087615,3.708193e+00,0.000000,79.923696,5.534657e-01,0.000000,9.073721,12.781914,0.000000,47.499884,0.002134,0.002469,0.000000,NaN,2.994740,13.072588,9.388184
82453,2,WAS,WR,00-0037742,2025_14_WAS_MIN,2025,14,2025,Treylon Burks,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.461517,0.000000,0.000000,0.005183,0.000000,5.578635,1.590897e+00,0.000000,34.246732,1.410843e-03,0.000000,2.386481,3.977378,0.000000,27.735756,0.300139,0.220349,0.000000,NaN,2.704258,10.606373,9.614230


In [15]:
def stack_players(df, num_players, index_cols, value_cols):
    'df should be seperated by position already'
    df = df.loc[df.dense_depth < num_players]
    piv = df.pivot(index=index_cols, columns=['position','dense_depth'], values=value_cols).reset_index()
    piv.columns = [x[0] + '_' + x[1] +'_' +  (str(int(x[2]))) if x[2] != '' else x[0] + x[1] for x in piv.columns ]
    return piv

In [16]:
dup_rows = agg[agg.duplicated(subset=['position','dense_depth'] + ['team','season','week'], keep=False)]
#dup_rows = dup_rows.loc[dup_rows.position == 'WR']
dup_rows[['gsis_id','game_id','player_display_name','dense_depth', 'position', 'team']].sort_values(by=['position','dense_depth'])

,gsis_id,game_id,player_display_name,dense_depth,position,team


In [17]:
test = df.loc[df.position == 'QB'].copy()
test.loc[test.game_id == '2024_04_TEN_MIA'][['gsis_id','game_id','player_display_name','dense_depth', 'position', 'team']].sort_values(by=['team','dense_depth'])

,gsis_id,game_id,player_display_name,dense_depth,position,team
72085,00-0035993,2024_04_TEN_MIA,Tyler Huntley,0,QB,MIA
72048,00-0039152,2024_04_TEN_MIA,Will Levis,0,QB,TEN
72049,00-0034771,2024_04_TEN_MIA,Mason Rudolph,1,QB,TEN


In [18]:
cols_to_agg =  (['gsis_id','player_display_name'] + roll_columns)

In [19]:
qb_roll_stat_cols = ['gsis_id','player_display_name'] + [x + '_roll' for x in qb_stats_eng] 
receiver_roll_stat_cols = ['gsis_id','player_display_name'] + [x + '_roll' for x in receiver_stats_eng] 
rush_stat_cols = ['gsis_id','player_display_name'] + [x + '_roll' for x in rush_stats_eng]

team_qb_stats_raw = stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team','season','week'], qb_roll_stat_cols)
wr_temp = stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team','season','week'], receiver_roll_stat_cols)
te_temp = stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],3,  ['team','season','week'], receiver_roll_stat_cols)
rb_temp =stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],2,  ['team','season','week'], receiver_roll_stat_cols)
team_receiver_stats_raw = wr_temp.merge(te_temp, on=['team','season','week'])
team_receiver_stats_raw = team_receiver_stats_raw.merge(rb_temp, on=['team','season','week'])
team_rb_stats_raw = stack_players(agg.loc[(agg.position.isin(['RB'])) & (~agg.week.isna())],3,  ['team','season','week'], rush_stat_cols)

In [20]:
stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],5,  ['team','season','week'], receiver_roll_stat_cols)

,team,season,week,gsis_id_WR_2,gsis_id_WR_3,gsis_id_WR_0,gsis_id_WR_1,gsis_id_WR_4,player_display_name_WR_2,player_display_name_WR_3,player_display_name_WR_0,player_display_name_WR_1,player_display_name_WR_4,receptions_roll_WR_2,receptions_roll_WR_3,receptions_roll_WR_0,receptions_roll_WR_1,receptions_roll_WR_4,targets_roll_WR_2,targets_roll_WR_3,targets_roll_WR_0,targets_roll_WR_1,...,fantasy_points_ppr_roll_WR_1,fantasy_points_ppr_roll_WR_4,offense_snaps_roll_WR_2,offense_snaps_roll_WR_3,offense_snaps_roll_WR_0,offense_snaps_roll_WR_1,offense_snaps_roll_WR_4,offense_pct_roll_WR_2,offense_pct_roll_WR_3,offense_pct_roll_WR_0,offense_pct_roll_WR_1,offense_pct_roll_WR_4,yards_per_catch_roll_WR_2,yards_per_catch_roll_WR_3,yards_per_catch_roll_WR_0,yards_per_catch_roll_WR_1,yards_per_catch_roll_WR_4,yards_per_target_roll_WR_2,yards_per_target_roll_WR_3,yards_per_target_roll_WR_0,yards_per_target_roll_WR_1,yards_per_target_roll_WR_4
0,ARI,2013,1,00-0027691,00-0030300,00-0022921,00-0029638,NaN,Andre Roberts,Jaron Brown,Larry Fitzgerald,Michael Floyd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARI,2013,2,00-0030300,00-0027691,00-0022921,00-0029638,NaN,Jaron Brown,Andre Roberts,Larry Fitzgerald,Michael Floyd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARI,2013,3,00-0027691,00-0030300,00-0022921,00-0029638,00-0028606,Andre Roberts,Jaron Brown,Larry Fitzgerald,Michael Floyd,Kerry Taylor,8.0,1.0,8.0,4.0,NaN,9.0,2.0,14.0,6.0,...,12.2,NaN,51.0,12.0,71.0,67.0,NaN,0.72,0.17,1.0,0.94,NaN,11.222222,9.0,9.333333,17.2,NaN,10.4,7.0,5.8,12.714286,NaN
3,ARI,2013,4,00-0027691,00-0030300,00-0022921,00-0029638,00-0028606,Andre Roberts,Jaron Brown,Larry Fitzgerald,Michael Floyd,Kerry,5.142857,1.0,4.571429,3.428571,NaN,8.428571,2.0,8.857143,6.571429,...,8.2,NaN,52.142857,12.571429,56.714286,67.0,NaN,0.737143,0.175714,0.8,0.94,NaN,10.767442,8.142857,10.25641,11.677419,NaN,7.333333,6.428571,6.101449,7.226415,NaN
4,ARI,2013,5,00-0027691,00-0030300,00-0029638,00-0022921,NaN,Andre Roberts,Jaron,Michael Floyd,Larry Fitzgerald,NaN,3.351351,NaN,3.675676,4.756757,NaN,6.513514,NaN,6.324324,7.621622,...,13.459459,NaN,43.0,NaN,61.810811,56.837838,NaN,0.647568,NaN,0.944324,0.877838,NaN,9.621118,NaN,11.179191,10.741784,NaN,5.971223,NaN,7.546125,7.520376,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6693,WAS,2025,9,00-0032398,00-0040164,00-0035719,00-0039355,NaN,Chris Moore,Jaylin Lane,Deebo Samuel Sr.,Luke McCaffrey,NaN,1.661708,1.698739,5.113512,0.900888,NaN,3.492983,3.412693,6.592711,1.723619,...,5.412067,NaN,42.69868,30.223639,46.292019,24.215052,NaN,0.683297,0.471602,0.746081,0.375389,NaN,13.478416,12.82954,8.398731,13.445837,NaN,8.652515,8.526218,7.15762,10.485692,NaN
6694,WAS,2025,10,00-0032398,00-0037742,00-0035719,00-0040164,NaN,Chris Moore,Treylon Burks,Deebo Samuel Sr.,Jaylin Lane,NaN,1.246281,1.160828,4.585134,1.504621,NaN,2.619737,2.191688,6.444534,3.020229,...,4.221163,NaN,37.77401,36.991188,44.469014,28.216821,NaN,0.619972,0.594045,0.744561,0.460045,NaN,12.423515,7.687686,7.566391,10.427114,NaN,8.538383,6.144638,6.079536,7.242365,NaN
6695,WAS,2025,11,00-0032688,00-0032398,00-0035719,00-0040164,NaN,Robbie Chosen,Chris Moore,Deebo Samuel Sr.,Jaylin Lane,NaN,1.160052,0.934711,4.68885,1.908814,NaN,1.771792,1.964803,6.3334,3.555351,...,4.674944,NaN,18.602415,40.330508,44.351761,31.67205,NaN,0.286733,0.649979,0.728421,0.505983,NaN,9.948407,11.335043,7.548886,9.6179,NaN,8.835108,8.408662,6.265099,6.800065,NaN
6696,WAS,2025,13,00-0037742,00-0040164,00-0035659,00-0035719,00-0032398,Treylon Burks,Jaylin Lane,Terry McLaurin,Deebo Samuel Sr.,Chris Moore,1.120567,1.668053,3.944258,4.516638,0.701033,1.893367,2.878391,6.50064,6.00005,...,11.378804,2.57

In [21]:
team_qb_stats_raw.to_csv('data/team_qb_stats.csv')
team_receiver_stats_raw.to_csv('data/team_receiver_stats.csv')
team_rb_stats_raw.to_csv('data/team_rb_stats.csv')

con.execute("CREATE OR REPLACE TABLE team_qb_stats AS SELECT * FROM team_qb_stats_raw")
con.execute("CREATE OR REPLACE TABLE team_receiver_stats AS SELECT * FROM team_receiver_stats_raw")
con.execute("CREATE OR REPLACE TABLE team_rb_stats AS SELECT * FROM team_rb_stats_raw")

In [23]:
qb_rush_stats =stack_players(agg.loc[(agg.position.isin(['QB'])) & (~agg.week.isna())],2,  ['team','season','week'], rush_stat_cols)
qb_rush_stats = qb_rush_stats.fillna(0)
te_rush_stats =stack_players(agg.loc[(agg.position.isin(['TE'])) & (~agg.week.isna())],1,  ['team','season','week'], rush_stat_cols)
te_rush_stats = te_rush_stats.fillna(0)
wr_rush_stats =stack_players(agg.loc[(agg.position.isin(['WR'])) & (~agg.week.isna())],1,  ['team','season','week'], rush_stat_cols)
wr_rush_stats = wr_rush_stats.fillna(0)

C:\Users\Sharedw\AppData\Local\Temp\ipykernel_40580\1118549396.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  qb_rush_stats = qb_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_40580\1118549396.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  te_rush_stats = te_rush_stats.fillna(0)
C:\Users\Sharedw\AppData\Local\Temp\ipykernel_40580\1118549396.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in

In [24]:
temp = team_rb_stats_raw.merge( qb_rush_stats,  on=['team','season','week'])
temp = temp.merge( te_rush_stats,  on=['team','season','week'])
temp = temp.merge(wr_rush_stats,  on=['team','season','week'])
temp.to_csv('data/team_rushers.csv')

con.execute("CREATE OR REPLACE TABLE team_rushers AS SELECT * FROM temp")

In [25]:
con.commit()
con.close()

In [26]:
team_qb_stats_raw

,team,season,week,gsis_id_QB_0,gsis_id_QB_1,player_display_name_QB_0,player_display_name_QB_1,completions_roll_QB_0,completions_roll_QB_1,attempts_roll_QB_0,attempts_roll_QB_1,passing_yards_roll_QB_0,passing_yards_roll_QB_1,passing_tds_roll_QB_0,passing_tds_roll_QB_1,passing_interceptions_roll_QB_0,passing_interceptions_roll_QB_1,sacks_suffered_roll_QB_0,sacks_suffered_roll_QB_1,sack_yards_lost_roll_QB_0,sack_yards_lost_roll_QB_1,passing_air_yards_roll_QB_0,passing_air_yards_roll_QB_1,passing_yards_after_catch_roll_QB_0,passing_yards_after_catch_roll_QB_1,passing_first_downs_roll_QB_0,passing_first_downs_roll_QB_1,passing_2pt_conversions_roll_QB_0,passing_2pt_conversions_roll_QB_1,carries_roll_QB_0,carries_roll_QB_1,fantasy_points_ppr_roll_QB_0,fantasy_points_ppr_roll_QB_1,offense_snaps_roll_QB_0,offense_snaps_roll_QB_1,offense_pct_roll_QB_0,offense_pct_roll_QB_1,completion_pct_roll_QB_0,completion_pct_roll_QB_1
0,ARI,2013,1,00-0021429,NaN,Carson Palmer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ARI,2013,2,00-0021429,NaN,Carson Palmer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ARI,2013,3,00-0021429,NaN,Carson Palmer,NaN,26.0,NaN,40.0,NaN,327.0,NaN,2.0,NaN,1.0,NaN,4.0,NaN,-23.0,NaN,385.0,NaN,61.0,NaN,19.0,NaN,0.0,NaN,0.0,NaN,17.08,NaN,71.0,NaN,1.0,NaN,0.65,NaN
3,ARI,2013,4,00-0021429,NaN,Carson Palmer,NaN,23.714286,NaN,39.428571,NaN,281.857143,NaN,1.428571,NaN,1.0,NaN,2.285714,NaN,-12.142857,NaN,382.714286,NaN,86.142857,NaN,15.0,NaN,0.0,NaN,1.142857,NaN,14.017143,NaN,71.0,NaN,1.0,NaN,0.601449,NaN
4,ARI,2013,5,00-0021429,NaN,Carson Palmer,NaN,21.243243,NaN,37.513514,NaN,240.837838,NaN,0.810811,NaN,1.432432,NaN,3.027027,NaN,-18.135135,NaN,368.567568,NaN,89.108108,NaN,12.405405,NaN,0.0,NaN,0.648649,NaN,9.460541,NaN,65.378378,NaN,1.0,NaN,0.566282,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6673,WAS,2025,9,00-0039910,00-0032268,Jayden Daniels,Marcus Mariota,20.104438,9.033932,31.405945,14.884077,222.835705,106.937958,1.878443,0.793032,0.426286,0.553877,2.591303,1.434572,-12.1755,-8.200986,241.601593,147.917528,101.915395,37.75073,10.937182,5.003798,0.226686,0.191063,9.315059,3.116688,20.793548,10.335226,66.22178,33.634331,0.975909,0.565904,0.640148,0.606953
6674,WAS,2025,10,00-0032268,00-0026300,Marcus Mariota,Josh Johnson,12.025449,6.331164,18.663058,10.130615,133.453468,66.543072,0.844774,0.411981,0.915408,0.260674,1.575929,1.021646,-9.40074,-7.009514,151.438146,78.108075,58.813048,30.421885,6.002848,3.883654,0.143297,0.0,4.337516,2.365708,10.581419,4.051371,38.475748,21.810894,0.674428,0.373043,0.644345,0.624954
6675,WAS,2025,11,00-0032268,00-0026300,Marcus Mariota,Josh Johnson,9.019087,4.954774,13.997294,8.031468,100.090101,53.493956,0.63358,0.305616,0.686556,0.193374,1.181947,0.757879,-7.050555,-5.199811,113.57861,71.625719,44.109786,22.825797,4.502136,3.139158,0.107473,0.0,3.253137,1.754934,7.936064,3.17063,30.856811,19.019757,0.535821,0.320622,0.644345,0.61692
6676,WAS,2025,13,00-0032268,NaN,Marcus Mariota,NaN,10.764315,NaN,15.99797,NaN,128.317576,NaN,0.975185,NaN,0.514917,NaN,1.38646,NaN,-9.787916,NaN,157.183957,NaN,42.832339,NaN,5.626602,NaN,0.080605,NaN,3.689853,NaN,10.632048,NaN,35.892608,NaN,0.636866,NaN,0.672855,NaN


In [28]:
qb_roll_stat_cols

['gsis_id',
 'player_display_name',
 'completions_roll',
 'attempts_roll',
 'passing_yards_roll',
 'passing_tds_roll',
 'passing_interceptions_roll',
 'sacks_suffered_roll',
 'sack_yards_lost_roll',
 'passing_air_yards_roll',
 'passing_yards_after_catch_roll',
 'passing_first_downs_roll',
 'passing_2pt_conversions_roll',
 'carries_roll',
 'fantasy_points_ppr_roll',
 'offense_snaps_roll',
 'offense_pct_roll',
 'completion_pct_roll']